In [1]:
import sys
import os

sys.path.append('F:/ai-ml/itapia/backend/evo_worker')
sys.path.append('F:/ai-ml/itapia/backend/shared')

In [2]:
os.environ['POSTGRES_HOST'] = 'localhost'
os.environ['REDIS_HOST'] = 'localhost'

In [3]:
from app.dependencies import get_backtest_context_manager, create_dependencies
create_dependencies()

In [4]:
context_mng = get_backtest_context_manager()

In [5]:
context_mng.init_ready_contexts()

In [6]:
import nest_asyncio
nest_asyncio.apply()
await context_mng.get_context('AAPL').load_data_into_memory()

[2025-08-21 12:57:45] INFO by Backtest Context: Lazy loading data for ticker: AAPL
[2025-08-21 12:57:45] INFO by Backtest Data Preparer: Preparing daily OHLCV for ticker 'AAPL'...
[2025-08-21 12:57:45] INFO by Prices Service of DB: SERVICE: Preparing daily prices for ticker AAPL
[2025-08-21 12:57:45] INFO by Metadata Service of DB: SERVICE: Preparing ticker info metadata of ticker AAPL...
[2025-08-21 12:57:45] INFO by Backtest Data Preparer: Transforming single ticker repsonse ...
[2025-08-21 12:57:45] INFO by Backtest Context: Successfully loaded 107 reports and OHLCV data for AAPL.


In [7]:
test_ctx = context_mng.get_context('AAPL')

In [8]:
from app.backtest.evaluator import FitnessEvaluator
from app.backtest.action import ShortSwingActionMapper

In [9]:
action_mapper = ShortSwingActionMapper()

In [10]:
evaluator = FitnessEvaluator(test_ctx, action_mapper)

In [11]:
from itapia_common.dblib.services import RuleService
from itapia_common.dblib.session import get_rdbms_session

db = next(get_rdbms_session())
rule_service = RuleService(db)

In [12]:
rules = rule_service.get_all_active_rules()

In [13]:
test_rule_ent = rules[2]

In [14]:
from itapia_common.rules.rule import Rule

test_rule = Rule.from_entity(test_rule_ent)

In [18]:
obj, metrics = evaluator.evaluate(test_rule)

[2025-08-21 14:17:52] INFO by Backtest Simulator: Running simulation on 1919 historical data points...
[2025-08-21 14:17:53] INFO by Backtest Simulator: Simulation finished. Total trades executed: 24


In [19]:
metrics

BacktestPerformanceMetrics(num_trades=24, total_return_pct=0.07071921098612162, max_drawdown_pct=0.044534031673522365, win_rate_pct=0.625, profit_factor=1.6798676964213475, sharpe_ratio=3.5426853356497587)

In [20]:
obj

(0.07071921098612162,
 3.5426853356497587,
 0.9554659683264777,
 0.625,
 0.5187150381839054,
 24.0)

In [38]:
# Trong notebook của bạn, đảm bảo có các import này
from itapia_common.rules.rule import Rule
from itapia_common.rules.parser import parse_tree
from itapia_common.rules import names as nms
from itapia_common.rules.nodes.registry import create_node, get_nodes_by_type
from itapia_common.schemas.enums import NodeType, SemanticType
from itapia_common.schemas.entities.rules import RuleEntity, NodeEntity

In [30]:
sell_strong_node = create_node(nms.CONST_NUM(-0.8))

In [36]:
get_nodes_by_type(node_type=NodeType.OPERATOR, purpose=SemanticType.DECISION_SIGNAL)

[NodeSpecEntity(node_name='OPR_TO_DECISION_SIGNAL', description='Scales any numerical input to [-1, 1] and casts it as a DECISION_SIGNAL.', node_type=<NodeType.OPERATOR: 'operator'>, return_type=<SemanticType.DECISION_SIGNAL: 'DECISION_SIGNAL'>, args_type=[<SemanticType.ANY: 'ANY'>])]

In [39]:
from datetime import datetime, timezone
rule_ent = RuleEntity(
    rule_id='SELL_OVER',
    name='sell',
    description='',
    purpose=SemanticType.DECISION_SIGNAL,
    version=1.0,
    is_active=True,
    created_at=datetime.now(timezone.utc),
    updated_at=datetime.now(timezone.utc),
    root=NodeEntity(
        node_name=nms.OPR_TO_DECISION_SIGNAL,
        children=[
            NodeEntity(node_name=nms.CONST_NUM(-0.8))
        ]
    )
)

In [40]:
test_rule = Rule.from_entity(rule_ent)

-0.8

In [46]:
obj, metrics1 = evaluator.evaluate(test_rule)

[2025-08-21 14:29:33] INFO by Backtest Simulator: Running simulation on 1919 historical data points...
[2025-08-21 14:29:33] INFO by Backtest Simulator: Simulation finished. Total trades executed: 92


In [47]:
from app.backtest.evaluator import metrics

In [50]:
metrics.trades_df['action_type'].value_counts()

action_type
SHORT    92
Name: count, dtype: int64